# DAX expansion Market activity - Python

### Overview
This notebook is the basis of this blog post : https://ganymde.cloud/dax-expansion.html

In this sample, we will get an insight on how an index impacts a stock liquidity. We will look back on how DAX index expansion from 30 to 40 stocks impacted liquidity for these new stocks leveraging several APIs we provide.

Following the announcement of Deutsche Böerse in 2020, the index became larger since <i>20 September 2021</i> by incorporating ten of the largest companies from Germany's MDAX listing 👇
* *Airbus SE, Brenntag SE, HelloFresh SE, Porsche SE, Puma SE, Qiagen NV, Sartorius AG (Pref. shares), Siemens Healthineers AG, Symrise AG and Zalando SE*

In order to understand the **index expansion impact on liquidity** for DAX components, we suggest to start with a purely technical indicator which is the **ticks count**, internally named `topolgy`. Each **market event** is pre-mapped, normalized and stored as a *tick* in our data store:
* Trades: timestamps, prices, sizes, trades Ids, trade conditions, etc.
* Book quotes (best bids and asks): timestamps, prices, sizes, etc.

This samples enables to retrieve **on-demand** ticks count data points by calling a dedicated service.

### Inputs/outputs
Tick count sample requires instruments' identifiers, date time intervals and time granularity as per inputs. It returns the ticks count data sampled using the input time granularity, readily available for on-the-fly analytics.

We will request **ticks count for the DAX components** before and after the index expansion, on *20 September 2021*. 
Then, we will compute **market activity movements** for each new component in order to understand the `integration into index impact on liquidity`.

### Services used
This sample uses *gRPC requests* in order to retrieve ticks from the dedicated hosted service. The queried endpoint in this script are:
* TopologiesService: to directly retrieve ticks objects from the server.

### Modules required
1. Systemathics packages:
    * *systemathics.apis.services.topology.v1*
    * *systemathics.apis.type.shared.v1*
    * *google.type*
2. Open source packages
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    * *pandas*
    * *matpotlib* as per display package
    
***

# Run DAX expansion - Market activity

### Step 1: Import required packages

In [ ]:
pip install googleapis-common-protos protobuf grpcio pandas matplotlib systemathics.apis --quiet

In [ ]:
import os
import grpc
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date
from datetime import datetime
import google.type.date_pb2 as date
import systemathics.apis.type.shared.v1.level_pb2 as level
import systemathics.apis.type.shared.v1.identifier_pb2 as identifier
import systemathics.apis.services.topology.v1.topologies_pb2 as topologies
import systemathics.apis.services.topology.v1.topologies_pb2_grpc as topologies_service

### Step 2: Prepare API requests
The following code snippets retrieve authentication token to be used in upcomming API requests:

In [ ]:
token = f"Bearer {os.environ['AUTH0_TOKEN']}"
display(token)

### Step 3: Prepare request parameters

#### 3.1 Instruments' selection
The following code snippet enables to set the **DAX components**, before and after index expansion. 
As we are using *ICE data services* as market data source, tickers are simply the `ICE symbols`.

In [ ]:
old_components = ['E:ADSD', 'E:ALVD', 'E:BASD', 'E:BAYND', 'E:BEID', 'E:BMWD', 'E:CBKD', 'E:COND', 'E:DAID', 'E:DBKD', 'E:DB1D', 'E:LHAD', 'E:DPWD', 'E:DTED', 'E:EOAND', 'E:FRED', 'E:FMED', 'E:HEID', 'E:HEND', 'E:IFXD', 'E:LIND', 'E:MRKD', 'E:MUV2D', 'E:PSMD', 'E:RWED', 'E:SAPD', 'E:SIED', 'E:TKAD', 'E:VOWD', 'E:VNAD']
new_components = ['E:PUMD', 'E:ZALD', 'E:SHLD', 'E:SY1D', 'E:HFGD', 'E:SRTD', 'E:PAH3D', 'E:BNRD', 'E:QIAD', 'E:AIRP']

Each ticker is available in a specific ICE data location called `ICE data source` with a **source_Id**.

In [ ]:
source = '840' # ICE data location for DAX tickers after source_Id migration  occured on "date to be added"

#### 3.2 Topologies parameters
The following code snippet enables to set the required parameters to call **TopologiesService**.

In [ ]:
# set topology time granularity (daily, weekly...)
granularity = topologies.TOPOLOGY_GRANULARITY_DAILY

# set level: Trades or Trades and Book
level = level.LEVEL_TRADES_AND_BOOK

### Step 4: Retrieve data
The following code snippet creates a method to handle request creation to **TickTradesService** for a given instrument with previous chosen parameters.

In [ ]:
# define method to handle topologies request creation for each instrument
def get_topologies_request(ticker, exchange, granularity, level):
    request = topologies.TopologiesRequest(identifier = identifier.Identifier(exchange = exchange, ticker = ticker),
                                           granularity = granularity,
                                           level = level)
    return request

The following code snippet creates a method that calls **TickTradesService** and returns *a pandas dataframe* containing **ticks counts** for a given list of instruments.

In [ ]:
# define method to handle tick counts data for a given list of instruments (old/new DAX components)
def get_toplogies_data(tickers):
    ticks_data = pd.DataFrame({'Date': []})
    ticks_data = ticks_data.set_index('Date')

    # iterate all instrument identifiers: exhange/ticker pairs
    for i in range(len(tickers)):
        ticker = tickers[i]
        
        # open a gRPC channel
        with open(os.environ['SSL_CERT_FILE'], 'rb') as f:
            credentials = grpc.ssl_channel_credentials(f.read())
        with grpc.secure_channel(os.environ['GRPC_APIS'], credentials) as channel:

            # instantiate the topologies service
            request = get_topologies_request(ticker, source, granularity, level)
            service = topologies_service.TopologiesServiceStub(channel)

            # process the topologies request
            response = service.Topologies(request=request, metadata = [('authorization', token)])

            # build data structure with: ticker, date and ticks count
            dates, ticks = [], []
            for entry in response.entries:
                if entry.begin.year >= 2021 and entry.begin.month >= 6 and entry.begin.day >= 1:
                    dates.append(datetime(year=entry.begin.year,day=entry.begin.day, month=entry.begin.month))
                    ticks.append(entry.ticks_count)

            tmp_ticks_data = pd.DataFrame(data ={'Date': dates, f'{ticker}': ticks})
            tmp_ticks_data = tmp_ticks_data.set_index('Date')
            if (ticks_data.size == 0):
                ticks_data = tmp_ticks_data
            else:
                ticks_data = pd.merge(ticks_data, tmp_ticks_data, on="Date")
                
    return ticks_data

#### 4.1 DAX previous composition
The following code snippet calls the previous method and builds a *pandas dataframe* with `ticks counts for the DAX previous composition, total of 30 instruments`.

In [ ]:
# get ticks count for the DAX previous composition
#old_topologies = get_toplogies_data(old_components)

In [ ]:
# display ticks count for the DAX previous composition
#old_topologies

#### 4.2 DAX new components
The following code snippet calls the previous method and builds a *pandas dataframe* with `ticks counts for the DAX new components, total of 10 instruments`.

In [ ]:
# get ticks count for the DAX new components
new_topologies = get_toplogies_data(new_components)

In [ ]:
# display ticks count for the DAX new components
new_topologies

#### 4.3 Market activity analysis following DAX expansion

In [ ]:
names = ['Puma SE', 'Zalando SE', 'Siemens Healthineers AG', 'Symrise AG', 'HelloFresh SE', 'Sartorius AG', ' Porsche Automobil Holding SE', 'Brenntag SE', 'QIAGEN NV', 'Airbus SE']

In [ ]:
# we start from June 1st until December 2nd , including September 20th, DAX expansion date
start_date = "2021-06-01"
end_date = "2021-12-02"
change_date = "2021-09-20"
day_before_change = "2021-09-19"

# compute ticks count percentage change
ratios = (new_topologies[change_date:end_date].mean() - new_topologies[start_date:day_before_change].mean()) / new_topologies[start_date:day_before_change].mean()
liquidity_ratios = round(ratios*100, 2)

# format data for display purposes
liquidity = pd.DataFrame({'Name': names,'Ticker': liquidity_ratios.index, 'Liquidity movement (%)': liquidity_ratios.values})
liquidity

### Step 5: Visualize data